In [691]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint
import matplotlib

from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

from src.UVMappingPipeline import UVMappingPipeline
from utils.sys_utils import *


pipe = UVMappingPipeline()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [798]:
%reload_ext autoreload
plt.close('all')
tdir = "/home/hunter/data/images/obstacles"
# tdir = "/home/hyoung/data/images/obstacles"
# tdir = homeDir + "/test/iai/obstacles"

found_imgs, _ = grab_dir_images(tdir)


p = interactive(pipe.pipeline,
    _img = widgets.Select(
                        options=sorted(found_imgs),
                        value=str(sorted(found_imgs)[0]),
                        description='File:',
                        layout=Layout(width='100%')
                    ),
    line_method = {'Standard Hough Transform':0,'Probabilistic Hough Transform':1},
    input_method = {'Classic Thresholded':0,'Morphed Thresholded':1,'Canny Edges':2},
    e1 = (1,255,1), e2 = (1,255,1),
    ang = (0,360,1), rho = (1,1000,1),
    minLineLength = (0,255,1), maxLineGap = (0,255,1),
    houghThresh = (0,255,1), greyThreshU = (0,255,1), greyThreshV = (0,255,1), cnt_thresh = (0,5000,100),
    show_helpers = True, use_umap = True, flip_thresh_bin = False, norm_grey_thresh = (0.0,1.0,0.01)
)


row1 = HBox([p.children[0]])
row2 = HBox([p.children[1], p.children[2]], layout = Layout(flex_flow='row wrap'))
row3 = HBox([p.children[3], p.children[4], p.children[9]], layout = Layout(flex_flow='row wrap'))
row4 = HBox([p.children[5], p.children[6], p.children[7], p.children[8]], layout = Layout(display='flex',flex_flow='row'))

row5 = HBox([p.children[10], p.children[11], p.children[12], ], layout = Layout(flex_flow='row wrap'))
row6 = HBox([p.children[16]], layout = Layout(flex_flow='row wrap'))
row7 = HBox([p.children[13], p.children[14], p.children[15]], layout = Layout(flex_flow='row wrap'))
controls = VBox([row1,row2,row3,row4,row5,row6,row7], layout = Layout(display='flex'))
output = p.children[-1]
display(VBox([controls, output]))

VkJveChjaGlsZHJlbj0oVkJveChjaGlsZHJlbj0oSEJveChjaGlsZHJlbj0oU2VsZWN0KGRlc2NyaXB0aW9uPXUnRmlsZTonLCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJScpLCBvcHRpb27igKY=


In [794]:
# im1 = pipe.umap; im2 = pipe.testU
im1 = pipe.vmap; im2 = pipe.testV

plt.figure(6)
plt.imshow(im1,'gray')
plt.show()

plt.figure(7)
plt.imshow(cv2.cvtColor(im2, cv2.COLOR_GRAY2BGR))
plt.show()

In [762]:
tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

normImg = (tmp1 - np.mean(tmp1))/np.std(tmp1)

dmax_norm = np.max(normImg)
dmin_norm = np.min(normImg)
test = cv2.normalize(normImg,None,0,1,cv2.NORM_MINMAX)
m = 255.0/(dmax_norm-dmin_norm)
print(dmin_norm,dmax_norm,m)
tmp = cv2.convertScaleAbs(test, test)#, m)#,np.mean(tmp))
print(np.min(tmp),np.max(tmp),np.mean(tmp))

fig1 = plt.figure(6)
fig1.canvas.set_window_title('Custom Filter Input')
plt.imshow(pipe.vmap,cmap='gray')
plt.show()

iters = 3; kSz = (1,6)
thresh = [5, 10, 20, 30]
filters = [cv2.THRESH_BINARY,cv2.THRESH_BINARY_INV,cv2.THRESH_TRUNC+cv2.THRESH_BINARY_INV,cv2.THRESH_TOZERO]
kernelC = cv2.getStructuringElement(cv2.MORPH_CROSS,kSz)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))

_, grey = cv2.threshold(tmp1,5,255,filters[0])#+cv2.THRESH_TRUNC)

# gradient = cv2.morphologyEx(grey, cv2.MORPH_CLOSE, kSz)
gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)

# tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
# preMorph = cv2.erode(grey,kernelE,iterations = 3)
# preMorph = cv2.dilate(grey,tmpKernel,iterations = 3)


# # morph = cv2.morphologyEx(grey,cv2.MORPH_CLOSE,(1,2), iterations = 2)
# morph = cv2.morphologyEx(grey,cv2.MORPH_OPEN,(2,3), iterations = 1)

# _,preMorph = cv2.threshold(preMorph,1,255,cv2.THRESH_BINARY_INV)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,15))
# preMorph = cv2.dilate(preMorph,tmpKernel,iterations = 3)
# tmpKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,3))
# preMorph = cv2.erode(preMorph,kernelE,iterations = 2)
# print(preMorph.shape)

# mask = cv2.cvtColor(preMorph,cv2.COLOR_BGR2GRAY)
# res = cv2.bitwise_and(tmp,tmp, mask=mask)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res



fig2 = plt.figure(7)
fig2.canvas.set_window_title('Custom Filter Output')
plt.imshow(out,cmap='gray')
plt.show()

(-0.15973804615444256, 23.061089565845283, 10.981520739089627)
(0, 1, 0.0021565755208333335)


In [ ]:
print("""
POTENTIALLY GOOD VMAP FILTERING
""")

tmp1 = np.copy(pipe.vmap); tmp0 = cv2.cvtColor(tmp1,cv2.COLOR_GRAY2BGR)

kSz = (1,6)
kernelE = cv2.getStructuringElement(cv2.MORPH_RECT,(1,15))
_, grey = cv2.threshold(tmp1,5,255,cv2.THRESH_BINARY)

gradient = cv2.morphologyEx(grey, cv2.MORPH_OPEN, kSz)
tmpKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,kSz)

gradient2 = cv2.erode(tmp0,kernelE,iterations = 1)
gradient2 = cv2.dilate(gradient2,tmpKernel,iterations = 1)
_, res = cv2.threshold(gradient2,5,255,cv2.THRESH_BINARY)

out = res

In [647]:
orig = np.copy(pipe.img)

imgMean = np.mean(orig)
imgStd = np.std(orig)
dmax_int = np.max(orig)
print('Original Image', imgMean, imgStd, dmax_int)

t0 = time.time()
normImg = (orig - np.mean(orig))/np.std(orig)
t1 = time.time()
dt1 = t1 - t0

# normImg = np.uint8(normImg)

dmax_norm1 = np.max(normImg)
dmin_norm1 = np.min(normImg)

m = 255.0/(dmax_norm1-dmin_norm1)
# m = 255.0/(dmax_norm1)
print(dmin_norm1,dmax_norm1,m)
# print(np.ceil(m))
normImg = cv2.convertScaleAbs(normImg, normImg, m,imgMean)
normMean = np.mean(normImg)
normStd = np.std(normImg)

print('Standardized Image #1', normMean, normStd, np.max(normImg), dt1)

t0 = time.time()
normImg2 = cv2.normalize(orig, orig, 0, 255, cv2.NORM_MINMAX)
t1 = time.time()
dt2 = t1 - t0

normMean2 = np.mean(normImg2)
normStd2 = np.std(normImg2)

print('Standardized Image #2', normMean2, normStd2, np.max(normImg2), dt2)

('Original Image', 89.217919921875, 104.29694599483896, 255)
(-0.8554221705234769, 1.5895199854301445, 104.29694599483896)
('Standardized Image #1', 89.217919921875, 104.29694599483896, 255, 0.0031609535217285156)
('Standardized Image #2', 89.217919921875, 104.29694599483896, 255, 0.00011801719665527344)


In [641]:
# umap,vmap,overlay,normImg  = get_uv_map(orig)
umap,vmap,overlay  = pipe.get_uv_map(normImg)

[UV Mapping] Input Image Size: (480, 640) --- w/ max disparity = 256.000
[UV Mapping] Disparity Map Sizes --- U-Map (256.00, 640.00) ||  V-Map (480.00, 256.00)
	[V Mapping] Scan [0] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [1] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [2] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [3] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [4] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [5] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [6] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [7] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [8] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [9] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [10] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [11] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [12] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [13] (640) ---- Scan Histogram (256, 1)
	[V Mapping] Scan [14] (640) ---- Scan

In [642]:
plt.figure(6)
plt.imshow(orig,'gray')
plt.show()

plt.figure(7)
plt.imshow(normImg, 'gray')
plt.show()

plt.figure(8)
plt.imshow(normImg2, 'gray')
plt.show()

plt.figure(9)
plt.imshow(umap, 'gray')
plt.show()

In [ ]:
print(pipe.nObs)
idx = 0

plt.figure(8)
plt.imshow(pipe.obsImgs[idx])
plt.show()

plt.figure(2)
plt.imshow(pipe.display_obstacles)
plt.show()

In [ ]:
# print(pipe.disparityBounds[obsId],pipe.xBounds[obsId])

tmp = np.copy(pipe.greyV)
tmp2 = np.copy(pipe.img)
tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
nObs = len(pipe.disparityBounds)
print(nObs)
b = 2; d = np.ceil(nObs / float(b))

plt.close(7)
fig, axes = plt.subplots(nrows=b, ncols=int(d), num=7)
obstacleImgs = []
for i in range(nObs):
    tmp2 = np.copy(pipe.img)
    tmp2 = cv2.cvtColor(tmp2,cv2.COLOR_GRAY2BGR)
    try:
        xs = pipe.xBounds[i]
        ys,disp = pipe.find_obstacles(tmp,pipe.disparityBounds[i], _threshold=30, lower_threshold=1, verbose = True)
        cv2.rectangle(tmp2,(xs[0],ys[0]),(xs[1],ys[-1]),(150,0,0),1)
    except:
        pass
    fig.axes[i].imshow(disp)
    obstacleImgs.append(tmp2)
plt.show()

In [ ]:
%reload_ext autoreload
# plt.figure(7)
h,w = pipe.vmap.shape[:2]
vmask = np.zeros((h,w,3), dtype=np.uint8)
cv2.rectangle(vmask,(0,0),(5,h),(255,255,255), cv2.FILLED)

mask = cv2.cvtColor(vmask,cv2.COLOR_BGR2GRAY)
mask_inv = cv2.bitwise_not(mask)

tmpV = np.copy(pipe.vmap)
tmpV = cv2.cvtColor(tmpV,cv2.COLOR_GRAY2BGR)
_, greyV = cv2.threshold(tmpV,11,255,cv2.THRESH_BINARY)

maskV, invMaskV = pipe.get_vmap_mask(greyV,verbose=True)

VMASK = cv2.bitwise_or(maskV, mask)
cv2.rectangle(invMaskV,(0,0),(5,h),(0,0,0), cv2.FILLED)

res = cv2.bitwise_and(pipe.vmap, pipe.vmap, mask = invMaskV)



_, greyV = cv2.threshold(res,11,255,cv2.THRESH_BINARY)

# plt.imshow(greyV)

cntsV, hierV = pipe.find_contours(greyV,20.0)
pipe.contoursV = cntsV; pipe.contour_hierarchyV = hierV

_cnts, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds(cntsV)
# x,y,w,h = cv2.boundingRect(cntsV[-3])
# print(x, x+w, y, y+h)
# cv2.rectangle(greyV,(x,y),(x+w,y+h),(150,0,0),1)

pipe.plot_contours(greyV,cntsV,ellipses,centers,pts)

# plt.show()

In [ ]:
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(pipe.dilatedU,kernel,iterations = 1)

comp_imgs = [pipe.umap,pipe.greyU,pipe.dilatedU,pipe.blurredU,pipe.closingU,erosion]
composite = pipe.construct_composite_img(comp_imgs,2,3)
pipe.plot_image(composite,3)

In [ ]:
posU = []
posV = []
def onclick(event):
    if event.dblclick:
        posU.append([event.xdata,event.ydata])
        print(posU)
#     print(event.ydata)

def onclickV(event):
    if event.dblclick:
        posV.append([event.xdata,event.ydata])
        print(posV)

In [ ]:
posU = []
posV = []
fu = plt.figure(6)
fu.canvas.mpl_connect('button_press_event', onclick)
plt.imshow(pipe.umap)
plt.show()

# fv = plt.figure(7)
# fv.canvas.mpl_connect('button_press_event', onclickV)

In [ ]:
plt.figure(7)
plt.imshow(pipe.vmap)
plt.show()

In [ ]:
# Image (frame_168 w/ 400 threshold disparities b/w 140 and 170)
cnts2, limits, ellipses, pts, centers = pipe.extract_contour_x_bounds()
print(limits)
# pp.pprint(hier)

temp = np.copy(pipe.img)
print(temp.shape)

goodxs = np.uint8(range(75,100))
goodpixels = np.uint8(range(140,425))

ix = np.isin(temp, goodxs)
ipx = np.isin(temp, goodpixels)
print(ix.shape,ipx.shape)

a = np.where(ix)
# b = np.isin(ix[1],range(140,425))
# c = np.where(b)

for i in range(len(a[0])):
    y = a[0][i]
    x = a[1][i]
    if(x>=140 and x <= 425):
#         print(x,y)
        temp[y,x] = (124)
    

print(a[1].shape)
# print(np.max(c[1]))
# print(np.max(a[1]))
# temp[c] = (123)

# print(a)

plt.figure(0)
plt.imshow(temp)
plt.show()

In [ ]:
x = np.arange(9.).reshape(3, 3)
print(x)
goodvalues = np.uint8(range(140,171))
print(goodvalues)
ix = np.isin(x, goodvalues)
a=np.where( x > 5 )
b = np.where( x < 7 )
c = np.bitwise_and(a,b)
print(a)
print(b)
print(c)


In [657]:
im = np.copy(pipe.img)
normIm = (im - np.mean(im))/np.std(im)
h, w = normIm.shape[:2]
plt.figure(0)
plt.imshow(im, cmap='gray')
plt.show()
plt.figure(1)
plt.imshow(normIm, cmap='gray')
plt.show()

In [685]:
dmax = np.max(normIm); dmin = np.min(normIm)
print dmax, dmin, (h,w)

ds = np.linspace(dmin, dmax,255)

print ds.shape
hu, wu = 255, w
hv, wv = h, 255

umap = np.zeros((hu,wu,1), dtype=np.uint8)
vmap = np.zeros((hv,wv,1), dtype=np.uint8)

1.5895199854301445 -0.8554221705234769 (480, 640)
(255,)


In [680]:
print umap.shape
us = normIm[:,0]
hist, edges = np.histogram(us,255,(dmin,dmax))
urow = np.reshape(hist, (hist.shape[0],1))

tmp = (urow - np.mean(urow))/np.std(urow)
print(us.shape, tmp.shape)

umap[:,0] = tmp


print(edges.shape)

(255, 640, 1)
((480,), (255, 1))
(256,)


In [686]:
for i in range(0,w):
    uscan = normIm[:,i]
    hist, edges = np.histogram(uscan,255,(dmin,dmax))
    urow = np.reshape(hist, (hist.shape[0],1))
    tmp = (urow - np.mean(normIm))/np.std(normIm)
    umap[:,i] = tmp
    
umap = np.reshape(umap,(hu,wu))

In [689]:
tmp = (pipe.umap - np.mean(pipe.umap))/np.std(pipe.umap)

plt.figure(0)
plt.imshow(pipe.umap, cmap='gray')
plt.show()


plt.figure(1)
plt.imshow(umap, cmap='gray')
plt.show()

plt.figure(2)
plt.imshow(tmp, cmap='gray')
plt.show()

In [ ]:
def test_uv_map(_img=None, verbose=True):
    overlay = []
        if(_img is None):
            img = np.copy(self.img)
            dmax = np.max(img) + 1
        else:
            img = np.copy(_img)
            dmax = np.max(img) + 1

        # Determine stats for U and V map images
        h, w = img.shape[:2]
        hu, wu = dmax, w
        hv, wv = h, dmax
        if(verbose):
            print("[UV Mapping] Input Image Size: (%d, %d) --- w/ max disparity = %.3f" % (h,w, dmax))
            print("[UV Mapping] Disparity Map Sizes --- U-Map (%.2f, %.2f) ||  V-Map (%.2f, %.2f)" % (hu, wu, hv, wv))

        histRange = (0,dmax)

        umap = np.zeros((hu,wu,1), dtype=self.img_dtype)
        vmap = np.zeros((hv,wv,1), dtype=self.img_dtype)

        if(timing): t0 = time.time()

        for i in range(0,h):
            vscan = img[i,:]
            vrow = cv2.calcHist([vscan],[0],None,[dmax],histRange)
            if(verbose): print("\t[V Mapping] Scan [%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, vscan.shape)), ', '.join(map(str, vrow.shape))))
            vmap[i,:] = vrow

        for i in range(0,w):
            uscan = img[:,i]
            urow = cv2.calcHist([uscan],[0],None,[dmax],histRange)
            if(verbose): print("\t[U Mapping] Scan[%d] (%s) ---- Scan Histogram (%s)" % (i,', '.join(map(str, uscan.shape)), ', '.join(map(str, urow.shape))))
            umap[:,i] = urow

        umap = np.reshape(umap,(hu,wu))
        vmap = np.reshape(vmap,(hv,wv))

        if(get_overlay):
            blank = np.ones((umap.shape[0],vmap.shape[1]),np.uint8)*255
            pt1 = np.concatenate((img, vmap), axis=1)
            pt2 = np.concatenate((umap,blank),axis=1)
            overlay = np.concatenate((pt1,pt2),axis=0)
            self.overlay = cv2.cvtColor(overlay,cv2.COLOR_GRAY2BGR)

        if(verbose): print("\t[UV Mapping] U Map = (%s) ----- V Map = (%s)" % (', '.join(map(str, umap.shape)),', '.join(map(str, vmap.shape)) ))
        self.umap = np.copy(umap)
        self.vmap = np.copy(vmap)
        if(timing):
            t1 = time.time()
            dt = t1 - t0
            print("\t[UV Mapping] --- Took %f seconds to complete" % (dt))
        return umap,vmap,overlay